In [ ]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/')
#jpred = pd.read_csv('./data/jpred4.tsv', sep='\t')
#jpred

#sns.boxplot(data=jpred, y= 'SCOPClass', x = 'Length')

In [ ]:
dssp = {'E': 0,
        'H': 0,
        '-': 0}

for filename in os.listdir('./data/dssp/'):
    file = open('./data/dssp/' + filename, 'r')
    file.readline()
    for line in file:
        dssp['H'] += line.count('H')
        dssp['E'] += line.count('E')
        dssp['-'] += line.count('-')

#plt.bar(dssp.keys(), dssp.values())
plt.pie(dssp.values(), labels=dssp.keys())

In [ ]:
# Comparative amino-acid composition of [countplot normalized]:
# The entire dataset
# The fraction of helix, strand and coil residue

amino = {}
for filename in os.listdir('./data/fasta/'):
    file = open('./data/fasta/' + filename, 'r')
    file.readline()
    for line in file:
        for aa in line.rstrip():
            if aa not in amino:
                amino[aa] = 1
            else:
                amino[aa] += 1

amino = pd.DataFrame.from_dict(amino, orient='index', columns = ['Counts'])

sns.set()
amino.reset_index(inplace= True)
sns.barplot(data = amino.sort_values('Counts'), x = 'index', y = 'Counts')

In [ ]:
helix = {}
strand = {}
coil = {}

for filename in jpred['DomainID']:
    dssp_file = open('./data/dssp/' + filename + '.dssp', 'r')
    fasta_file = open('./data/fasta/' + filename + '.fasta', 'r')
    dssp_file.readline(), fasta_file.readline()
    for dssp_line, fasta_line in zip(dssp_file, fasta_file):
        for secondary, aa in zip(dssp_line.rstrip(), fasta_line.rstrip()):
            if secondary == 'H':
                helix[aa] = helix.get(aa, 0) + 1
            elif secondary == 'E':
                strand[aa] = strand.get(aa, 0) + 1
            else:
                coil[aa] = coil.get(aa, 0) + 1
    dssp_file.close(), fasta_file.close()
    
coil

In [ ]:
secondary_structure = []
amino = []
count = []

for filename in jpred['DomainID']:
    dssp_file = open('./data/dssp/' + filename + '.dssp', 'r')
    fasta_file = open('./data/fasta/' + filename + '.fasta', 'r')
    dssp_file.readline(), fasta_file.readline()
    for dssp_line, fasta_line in zip(dssp_file, fasta_file):
        for (secondary, aa) in zip(dssp_line, fasta_line):
            #print(secondary, aa)
            if secondary in secondary_structure and aa in amino:
                if secondary_structure.index(secondary) == amino.index(aa): 
                    
                    count[secondary_structure.index(secondary)] +=1
            else:
                secondary_structure.append(secondary)
                amino.append(aa)
                count.append(0)
                    
    dssp_file.close(), fasta_file.close()

people = {"Structure": secondary_structure, "Aminoacid": amino, "Count": count}
people
#composition1 = pd.DataFrame(people) 
#composition1

In [ ]:
composition = pd.DataFrame([helix, strand, coil], index = ['Helix', 'Strand', 'Coil']).T

composition = composition.melt(ignore_index = False, value_name = 'Count', var_name = 'Structure')
composition.reset_index(level=0, inplace=True)
composition = composition[composition['index'] != '\n']
composition
plt.figure(figsize=(15,4))
sns.barplot(data=composition, hue='Structure', x='index', y='Count')

In [ ]:
# Taxonomic classification (at superkingdom [pie] and species level [pie or countplot,
# reducing the number of species to top 5/10/20])

jpred.Suprekingdom.value_counts().plot.pie()

In [ ]:
# Top x specie
jpred.TaxaName.value_counts()[0:10].plot.pie()

In [ ]:
# Structural classification (SCOP class)

jpred.SCOPClass.value_counts().plot.pie(autopct='%1.1f%%')

# Begin project code

In [ ]:
# Import the csv
import os
import pandas as pd
os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test/')
test = pd.read_csv('test.csv', usecols=[0,1,2,3,4,5])

In [ ]:
# Reduce internal redundancy

def reduce_internal_redundancy():
    test = test[test['Sequence'].str.contains('X') == False]
    test['Chain Length'] = pd.to_numeric(test['Chain Length'] , errors='coerce')
    test = test[test['Chain Length'].isna() == False]
    test = test.reset_index(drop = True)
    return test

In [ ]:
# Extract unclustered sequences to file

from os import path
if path.exists('test_unclustered.fasta'):
    os.remove('test_unclustered.fasta')
test_unclustered = open('test_unclustered.fasta', 'a')
for index in test.index:
        sequence = test.loc[index, 'Sequence']
        id = test.loc[index, 'PDB ID']
        chain = test.loc[index, 'Chain ID'].split(',')[0]
        print('>', id,'_', chain, '\n', sequence, sep='', end='\n', file = test_unclustered)
test_unclustered.close()

In [ ]:
# Run the clustering

%cd ~/PycharmProjects/lb2-2020-project-roncelli/data/test/
!mmseqs createdb test_unclustered.fasta db/testDB
!mmseqs cluster db/testDB test.clstr tmp/ --min-seq-id 0.3 -c 0.5 --cov-mode 0
!mmseqs createsubdb test.clstr db/testDB representatives
!mmseqs convert2fasta representatives representatives.fasta
!rm -r tmp/*
!rm test_unclustered.fasta 
!clear

In [ ]:
# Identify the sequences in the test set similar to the training

!blastp -query representatives.fasta -out representatives_similar.tsv -db db/jpred.fasta -outfmt 6 -evalue 0.01

In [ ]:
# Cleanup

to_keep = ['test.csv', 'representatives_similar.tsv', 'representatives.fasta']
for file in os.listdir('.'):
    if os.path.isfile(file) and file not in to_keep:
        os.remove(file)

In [ ]:
# Read the similar representatives from the blast output

representatives_similar = pd.read_csv('representatives_similar.tsv', 
                                      sep='\t',
                                      names=['Query', 
                                             'Subject', 
                                             'Identity', 
                                             'Length', 
                                             'mismatches', 
                                             'gap opens', 
                                             'Query start', 
                                             'Quesry end', 
                                             'Sequence start', 
                                             'Sequence end',
                                             'E-value', 
                                             'Bit score'])

In [ ]:
# Create the representatives dataframe

from Bio import SeqIO
import os

os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test')

representatives = list()
for record in SeqIO.parse("representatives.fasta", "fasta"):
    representatives.append([record.id, str(record.seq)])

representatives = pd.DataFrame(representatives, columns=['ID', 'Sequence'])    
similar_id = representatives_similar[representatives_similar['Identity'] > 30]
similar_id = similar_id['Query'].values.tolist()

In [ ]:
# Take a random sample

import random
test_list = random.sample(representatives['ID'].to_list(), k = 150)

In [ ]:
# Custom function to extract chains from a cif file 

import os
def extract_chain(file_path, out_path, chain):
    base_name = os.path.basename(file_path)
    name = os.path.splitext(base_name)[0]
    extension = '.cif'
    file = open(file_path + extension)
    if os.path.exists(out_path + name + '_' + chain + extension):
        os.remove(out_path + name + '_' + chain + extension)
    out = open(out_path + name + '_' + chain + extension, 'a')
    print(file.readline().rstrip() + '_' + chain + '\n#' + '\nloop_', end = '\n', file = out)
    counter = 0
    chain_index = 0
    for line in file:
        if line.startswith('_atom_site.'):
            while line.startswith('_atom_site.'):
                if 'auth_asym_id' in line:
                    chain_index = counter
                print(line, end = '', file = out)
                line = file.readline()
                counter += 1
        line2 = line.split()
        if len(line2) >= chain_index + 1 and line2[0] == 'ATOM' and line2[chain_index] == chain:
            print(line, end = '', file = out)
    print('#', file = out)
    out.close()

In [ ]:
# Download the structure file and extract the chains

from Bio.PDB import PDBIO
from Bio.PDB import FastMMCIFParser
from Bio.PDB import PDBList
from tqdm.notebook import tqdm
os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test/pdb')

pdbl = PDBList()
for record in tqdm(test_list):
    id = record[:4]
    chain = record[5:]
    #pdbl.retrieve_pdb_file(id, pdir = '.', file_format='mmCif')
    #os.rename(id.lower() + '.cif', id + '.cif')
    extract_chain(id.lower(), 'singlechains/', chain)
print('Done!')

In [ ]:
for filename in os.listdir('.'):
    os.rename(filename, filename.lower())

In [ ]:
# Custom function to remap secondary structures to simplified alphabet (C-H-E)

def ss_mapper(structure): 
    mapped_structure = []
    position = int()
    for secondary in structure:
        if secondary == ' ' or secondary == 'T' or secondary == 'S' or secondary == '-':
             secondary = 'C'
        elif secondary == 'H' or secondary == 'G' or secondary == 'I':
            secondary = 'H'
        elif secondary == 'B' or secondary == 'E':            
            secondary = 'E'
        else:
            if secondary == '\n':
                raise Exception('Character is a newline. Did you remember to rstrip?')
            else:
                raise Exception('Aminoacid in position ' + str(position) + ' is ' + "'" + secondary + "'")
        position += 1
        mapped_structure.append(secondary)
    mapped_structure = ''.join([str(elem) for elem in mapped_structure]) 
    return mapped_structure

In [ ]:
# Create the DSSP dataframe with Biopython

import pandas as pd
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from tqdm.notebook import tqdm

dssp_df = pd.DataFrame(columns=['PDB ID', "Sequence", "Secondary structure"])
os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test/pdb/singlechains/')
parser = MMCIFParser()

for pdb_id in tqdm(test_list):
    sequence = ''
    structure = ''

    structure = parser.get_structure(pdb_id, pdb_id.lower() + '.cif')
    dssp_dict = dssp_dict_from_pdb_file(pdb_id.lower() + '.cif')[0]
    entry_df = pd.DataFrame(dssp_dict).T
    entry_df.reset_index(drop = True)

    sequence = entry_df[0].str.cat()
    structure = ss_mapper(entry_df[1].str.cat())
    dssp_df = dssp_df.append({'PDB ID': pdb_id,
                   'Sequence': sequence,
                   'Secondary structure': structure}, ignore_index = True)

In [ ]:
# Run DSSP with shell

import os
os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test/pdb/singlechains/')

for pdb_id in tqdm(test_list):
    stream = os.popen('dssp -i ' + pdb_id.lower() + '.cif' + ' -o ../../dssp/' + pdb_id + '.dssp')

In [ ]:
# Create DSSP dataframe from output file created with Bash

import pandas as pd
from tqdm.notebook import tqdm

os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test/dssp')

dssp_df = pd.DataFrame(columns=['PDB ID', "Sequence", "Secondary structure"])
for pdb_id in test_list:
    if pdb_id + '.dssp' in os.listdir('.'):
        with open(pdb_id + '.dssp') as handle:
            sequence = list()
            structure = list()
            line = handle.readline().split()
            while line[0] != '#':
                line = handle.readline().split()
            for line in handle:
                line = list(line)
                if line[13] == '!':
                    sequence.append('X')
                    structure.append(' ')
                else:
                    sequence.append(line[13])
                    structure.append(line[16])
        sequence = ''.join(sequence)
        structure = ''.join(structure)
        structure = ss_mapper(structure)
        dssp_df = dssp_df.append({'PDB ID': pdb_id,
                                  'Sequence': sequence,
                                  'Secondary structure': structure}, ignore_index = True)
dssp_df

In [ ]:
a = ['5AJS_A', '4WA6_B']
dssp_df.query("Sequence in @test.Sequence")

In [ ]:
dssp_df[dssp_df['Sequence'] == 'RKCPSPNVIIITKPWEKGLKAITVAIRRNAIDSLPPNIKSLNYLNNILAKIEANAKGGDEAIFLDHNGYISEGSGDNIFIVKNGTITTPPTLNNLKGITRQVVIELINELEIPFREANIGLFDLYSADEIFVTGTAAEIAPVTYIDGRTVGNGKPGKVTKMLMEKFRERTENEGVEIYR']

In [ ]:
representatives[representatives['ID'] == '5CM0_A'].Sequence.item()

In [ ]:
for sequence in dssp_df.Sequence:
    len_dssp = int()
    len_test = int()
    sequence = sequence.replace('X', '')
    match = test.Sequence.str.match(sequence).any()
    if not match:
        truncated_sequence = representatives[representatives.Sequence.str.contains(sequence)]
    
        print(sequence)
        print(truncated_sequence)
        truncated_sequence = truncated_sequence.iloc[0]
        #print(sequence)
        #print(truncated_sequence['Sequence'], sep = '\n')
     #   found = test.loc[test['Sequence'].str.contains(sequence)]
        
      #  print(found.iloc[0, 2].find(sequence))
        #print(len(sequence), ' ', len(found['Sequence'].values[0]))
   # else:
        #print(sequence, 'OK!')

In [ ]:
for index1 in test.index:
    for index in dssp_df.index:
        if dssp_df.loc[index,"PDB ID"].split("_")[0]==test.loc[index1,"PDB ID"]:
            if dssp_df.loc[index,"PDB ID"].split("_")[1]==test.loc[index1,"Chain ID"].split(",")[0]:
                print(test.loc[index1,"PDB ID"])
                print("dssp")
                print(dssp_df.loc[index,"Sequence"])
                print("fasta")
                print(test.loc[index1,"Sequence"])

In [ ]:
# Run PSI blast

import os   
from Bio.Blast.Applications import NcbipsiblastCommandline
from tqdm.notebook import tqdm
for fasta_file in tqdm(os.listdir('training/fasta/')):
    break # to not run everything
    base_name = os.path.basename(fasta_file)
    name = os.path.splitext(base_name)[0]
    cmd = NcbipsiblastCommandline(query = 'training/fasta/' + base_name, 
                                  db = 'swiss-prot/uniprot_sprot.fasta', 
                                  evalue = 0.01,
                                  num_iterations = 3, 
                                  out_ascii_pssm = 'psiblast_output/' + name + '.pssm', 
                                  num_descriptions = 10000,
                                  num_alignments = 10000, 
                                  out= 'psiblast_output/' + name + '.alns.blast', 
                                  num_threads = 8)

    cmd()

In [10]:
# Create the profile files

import os
import time
import pandas as pd
from script.ss_mapper import ss_mapper
from tqdm.notebook import tqdm

os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/training/')
for sample in tqdm(os.listdir('pssm'), desc='Parsing'):
    time.sleep(0.01)
    base_name = os.path.basename(sample)
    name = os.path.splitext(base_name)[0]
    with open('pssm/' + sample, 'r') as pssm_file, open('dssp/' + name + '.dssp') as dssp_file:
            dssp_file.readline()
            structure = dssp_file.readline().rstrip()
            structure = ' ' + ss_mapper(structure)
            pssm_file.readline()
            pssm_file.readline()
            file_list = []
            offset = False
            position = 0
            for line in pssm_file:
                line = line.rstrip()
                if not line:
                    break
                line = line.split()
                line.append(structure[position])
                position += 1
                if not offset:
                    for i in range(2):
                        line.insert(0, '')
                        offset = True
                file_list.append(line)
            df = pd.DataFrame(file_list)
            #df.drop(df.tail(6).index, inplace=True)
            df.drop((df.columns[col] for col in range(2,22)), axis=1, inplace=True)
            df.drop((df.columns[-3:-1]), axis = 1,inplace = True)
            df.drop((df.columns[0]), axis = 1, inplace = True)
            df.columns = df.iloc[0] #grab the first row for the header
            df = df[1:] #take the data less the header row #set the header row as the df header
            df.rename(columns={ df.columns[0]: "Sequence" }, inplace = True)
            df.rename(columns={ df.columns[-1]: "Structure" }, inplace = True)
            df = df[ ['Structure'] + [ col for col in df.columns if col != 'Structure' ] ]
            df.loc[:, 'A':'V'] = df.loc[:, 'A':'V'].astype(float).divide(100)
            df.to_csv('profile/' + name + '.profile',sep='\t', index= False)

In [ ]:
# Removes profiles that are all zeroes

import os
import pandas as pd
os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/training/profile')
for file in os.listdir():
    df = pd.read_csv(file, sep = '\t')
    df = df.iloc[:, 2:]
    if not df.any(axis = None):
        os.remove(file)

In [ ]:
# Check if structure file has breaks

import os
os.chdir('/home/stefano/PycharmProjects/lb2-2020-project-roncelli/data/test/pdb/singlechains/')
def has_break(file_handle):
    base_name = os.path.basename(file_handle)
    name = os.path.splitext(base_name)[0]
    counter = 0
    residue_index = 0
    latest_numer = 0
    file = open(file_handle)
    residue_list = []
    for line in file:
        if line.startswith('_atom_site.'):
             while line.startswith('_atom_site.'):
                if 'auth_seq_id' in line:
                    residue_index = counter
                line = file.readline()
                counter += 1
        line = line.split()

        if len(line) > residue_index and line[0] == 'ATOM':
            residue_list.append(line[residue_index])

    for i in range(len(residue_list) - 1):
        if int(residue_list[i+1]) > int(residue_list[i]) + 1:
            break_length = int(residue_list[i +1]) - int(residue_list[i]) - 1
            print(base_name, 'Found break! Residues ', residue_list[i], '-', residue_list[i + 1], ' Break length: ', break_length)    

for file in os.listdir('.'):
    has_break(file)